# Análise de Jogos da NFL com Regressão Logística

Este notebook utiliza a regressão logística para prever o vencedor de jogos da NFL com base em vitórias e derrotas acumuladas dos times. A seguir, apresentamos o passo a passo do código:

---

## 1. Importação de Bibliotecas

Importamos as bibliotecas necessárias para manipulação de dados, divisão de conjuntos, criação do modelo e avaliação de desempenho.

## 2. Carregamento do dataset

Utilizei o datset "games.csv", que contém dados de jogos da NFL de 1999 até os jogos atuais, o dataset é atualizado toda semana e eu atualizo através desse repositório: "https://github.com/nflverse/nfldata/blob/master/data/games.csv"

## 3. Filtragem dos dados
Filtrei apenas os jogos de temporada regular, de 2013 pra frente

## 4. Criação de features

Vitória e derrotas acumuladas no time de casa e fora

## 5. Divisão em treino e teste

Utilizei 80% dos dados para treino e 20% para teste

## 6. Treinamento do modelo

Criei um moelo de regressão logística, que visa predizer o vencedor da partida

## 7. Avaliação do modelo

Calcula a acurácia do modelo

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from rich.console import Console
from rich.table import Table

In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Carregar dataset
df = pd.read_csv('games.csv')

# Filtrar temporadas a partir de 2013 e jogos da temporada regular
df = df[df['season'] >= 2013]
df = df[df['game_type'] == 'REG']

# Ordenar por temporada, data e hora dos jogos (respeitar ordem cronológica)
df = df.sort_values(by=['season', 'gameday'])

# Substituir valores NaN em colunas relevantes
colunas_para_corrigir = ['home_rest', 'away_rest', 'home_moneyline', 'away_moneyline', 
                         'home_spread_odds', 'away_spread_odds', 'home_score', 'away_score']
for coluna in colunas_para_corrigir:
    df[coluna] = df[coluna].fillna(0)

# Definir o vencedor (1 se o time da casa vencer, 0 se o time visitante vencer)
df['winner'] = df['result'].apply(lambda x: 1 if x > 0 else 0)
df = df[df['result'] != 0]  # Remover empates, se existirem

# Calcular estatísticas acumuladas respeitando a ordem cronológica
df['home_team_wins'] = df.groupby('home_team')['winner'].cumsum().shift(1).fillna(0)
df['away_team_wins'] = df.groupby('away_team')['winner'].transform(lambda x: (1 - x).cumsum()).shift(1).fillna(0)
df['home_team_losses'] = df.groupby('home_team')['winner'].transform(lambda x: (1 - x).cumsum()).shift(1).fillna(0)
df['away_team_losses'] = df.groupby('away_team')['winner'].cumsum().shift(1).fillna(0)

# Criar algumas features úteis
df['point_difference'] = df['home_score'] - df['away_score']
df['rest_difference'] = df['home_rest'] - df['away_rest']
df['moneyline_difference'] = df['home_moneyline'] - df['away_moneyline']
df['spread_odds_difference'] = df['home_spread_odds'] - df['away_spread_odds']

# Selecionar as colunas de interesse para o modelo
parametros = ['home_team_wins', 'away_team_wins', 'home_team_losses', 'away_team_losses',
              'point_difference', 'rest_difference', 'moneyline_difference', 'spread_odds_difference']

# Remover jogos com informações incompletas
df = df.dropna(subset=parametros)

# Dividir em treino e teste
df_train = df[df['season'] < 2023]
df_test = df[df['season'] >= 2023]

X_train = df_train[parametros]
y_train = df_train['winner']
X_test = df_test[parametros]
y_test = df_test['winner']

# Criar o modelo de regressão logística
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

# Fazer previsões
y_pred = modelo.predict(X_test)

# Avaliar a acurácia do modelo
acuracia = accuracy_score(y_test, y_pred)
print(f'Acurácia: {acuracia * 100:.2f}%')


Acurácia: 94.30%


In [3]:
df

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,...,stadium,winner,home_team_wins,away_team_wins,home_team_losses,away_team_losses,point_difference,rest_difference,moneyline_difference,spread_odds_difference
3714,2013_01_BAL_DEN,2013,REG,1,2013-09-05,Thursday,20:30,BAL,27.0,DEN,...,Sports Authority Field at Mile High,1,0.0,0.0,0.0,0.0,22.0,0,-599.0,219.0
3715,2013_01_NE_BUF,2013,REG,1,2013-09-08,Sunday,13:00,NE,23.0,BUF,...,Ralph Wilson Stadium,0,1.0,0.0,0.0,1.0,-2.0,0,781.0,-210.0
3716,2013_01_SEA_CAR,2013,REG,1,2013-09-08,Sunday,13:00,SEA,12.0,CAR,...,Bank of America Stadium,0,0.0,1.0,1.0,0.0,-5.0,0,286.0,-8.0
3717,2013_01_CIN_CHI,2013,REG,1,2013-09-08,Sunday,13:00,CIN,21.0,CHI,...,Soldier Field,1,0.0,1.0,1.0,0.0,3.0,0,-295.0,4.0
3718,2013_01_MIA_CLE,2013,REG,1,2013-09-08,Sunday,13:00,MIA,23.0,CLE,...,FirstEnergy Stadium,0,1.0,0.0,0.0,1.0,-13.0,0,-248.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6973,2024_18_MIA_NYJ,2024,REG,18,2025-01-05,Sunday,13:00,MIA,0.0,NYJ,...,MetLife Stadium,0,63.0,50.0,35.0,47.0,0.0,0,0.0,0.0
6974,2024_18_NYG_PHI,2024,REG,18,2025-01-05,Sunday,13:00,NYG,0.0,PHI,...,Lincoln Financial Field,0,40.0,39.0,58.0,59.0,0.0,0,0.0,0.0
6975,2024_18_CIN_PIT,2024,REG,18,2025-01-05,Sunday,13:00,CIN,0.0,PIT,...,Acrisure Stadium,0,61.0,33.0,36.0,65.0,0.0,4,0.0,0.0
6976,2024_18_NO_TB,2024,REG,18,2025-01-05,Sunday,13:00,NO,0.0,TB,...,Raymond James Stadium,0,66.0,42.0,31.0,55.0,0.0,0,0.0,0.0


In [4]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(modelo, X_train, y_train, cv=5)  # 5-fold cross-validation
print(f'Acurácia média com cross-validation: {scores.mean() * 100:.2f}%')


Acurácia média com cross-validation: 100.00%


In [9]:
# Instanciar o console do Rich
console = Console()

# Variável para o número da semana que você deseja visualizar
semana = int(input("Digite o número da semana (por exemplo, 1 para semana 1): "))

# Filtrar os jogos da temporada de 2024 para a semana desejada
df_semana = df_test[(df_test['season'] == 2024) & (df_test['week'] == semana)].copy()

# Fazer previsões de probabilidade
y_proba_semana = modelo.predict_proba(df_semana[parametros])

# Adicionar as colunas de probabilidade no DataFrame
df_semana['home_team_win_prob'] = y_proba_semana[:, 1]  # Probabilidade do time da casa vencer
df_semana['away_team_win_prob'] = y_proba_semana[:, 0]  # Probabilidade do time visitante vencer

# Adicionar a coluna de vencedor previsto
df_semana['predicted_winner'] = modelo.predict(df_semana[parametros])

# Salvar as previsões no arquivo CSV
arquivo_csv = f"previsões/Previsões_semana_{semana}.csv"
df_semana.to_csv(arquivo_csv, index=False)
console.print(f"[bold green]As previsões foram salvas no arquivo '{arquivo_csv}'.[/bold green]")

# Criar a tabela do Rich
table = Table(title=f"Predições para os Jogos da Semana {semana}")

# Adicionar colunas na tabela
table.add_column("Time da Casa", style="bold cyan")
table.add_column("Time Visitante", style="bold cyan")
table.add_column("Chance Casa (%)", style="green")
table.add_column("Chance Visitante (%)", style="red")
table.add_column("Vencedor Previsto", style="bold magenta")

# Adicionar as linhas de predições
for _, row in df_semana.iterrows():
    vencedor = row['home_team'] if row['predicted_winner'] == 1 else row['away_team']
    table.add_row(
        row['home_team'],
        row['away_team'],
        f"{row['home_team_win_prob']:.2%}",
        f"{row['away_team_win_prob']:.2%}",
        vencedor
    )

# Exibir a tabela
console.print(table)

Digite o número da semana (por exemplo, 1 para semana 1):  1


As previsões foram salvas no arquivo 'previsões/Previsões_semana_1.csv'.

                             Predições para os Jogos da Semana 1                              
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Time da Casa ┃ Time Visitante ┃ Chance Casa (%) ┃ Chance Visitante (%) ┃ Vencedor Previsto ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ KC           │ BAL            │ 100.00%         │ 0.00%                │ KC                │
│ PHI          │ GB             │ 100.00%         │ 0.00%                │ PHI               │
│ ATL          │ PIT            │ 0.00%           │ 100.00%              │ PIT               │
│ BUF          │ ARI            │ 100.00%         │ 0.00%                │ BUF               │
│ CHI          │ TEN            │ 100.00%         │ 0.00%                │ CHI               │
│ CIN          │ NE             │ 0.00%           │ 100.00%              │ NE                │
│ IND          │ HOU            │ 0.11%           │ 99.89%               │ HOU               │
│ MIA          │ JAX            │ 100.00%         │ 0.00%                │ MIA               │
│ NO           │ CAR            │ 100.00%         │ 0.00%                │ NO                │
│ NYG          │ MIN            │ 0.00%           │ 100.00%              │ MIN               │
│ LAC          │ LV             │ 100.00%         │ 0.00%                │ LAC               │
│ SEA          │ DEN            │ 100.00%         │ 0.00%                │ SEA               │
│ CLE          │ DAL            │ 0.00%           │ 100.00%              │ DAL               │
│ TB           │ WAS            │ 100.00%         │ 0.00%                │ TB                │
│ DET          │ LA             │ 100.00%         │ 0.00%                │ DET               │
│ SF           │ NYJ            │ 100.00%         │ 0.00%                │ SF                │
└──────────────┴────────────────┴─────────────────┴──────────────────────┴───────────────────┘

In [6]:
def calcular_lucro(row, valor_entrada):
    if row['predicted_winner'] == 1:  # Aposta no time da casa
        odds = row['home_moneyline']
        if row['winner'] == 1:  # Aposta certa (time da casa venceu)
            return valor_entrada * (odds / 100) if odds > 0 else valor_entrada * (100 / abs(odds))
        else:  # Aposta errada (time da casa perdeu)
            return -valor_entrada
    elif row['predicted_winner'] == 0:  # Aposta no time visitante
        odds = row['away_moneyline']
        if row['winner'] == 0:  # Aposta certa (time visitante venceu)
            return valor_entrada * (odds / 100) if odds > 0 else valor_entrada * (100 / abs(odds))
        else:  # Aposta errada (time visitante perdeu)
            return -valor_entrada
    else:  # Caso não haja previsão
        return 0

In [7]:
def moneyline_to_decimal(moneyline):
    if moneyline > 0:
        return (moneyline / 100) + 1
    elif moneyline < 0:
        return (100 / abs(moneyline)) + 1
    else:
        return None  # Odds inválidas

In [8]:
from rich.console import Console
from rich.table import Table# Instanciar o console do Rich
console = Console()

# Variável para o número da semana que você deseja visualizar
semana = int(input("Digite o número da semana (por exemplo, 1 para semana 1): "))

# Perguntar o valor da entrada nos jogos
valor_entrada = float(input("Digite o valor da entrada para cada jogo (exemplo: 100): "))

# Carregar as previsões para a semana escolhida
previsao = pd.read_csv(f'previsões/Previsões_semana_{semana}.csv')

# Filtrar os jogos da temporada de 2024 para a semana desejada
df_semana = df_test[(df_test['season'] == 2024) & (df_test['week'] == semana)].copy()

# Converter odds Moneyline para Decimais
df_semana['home_odds_decimal'] = df_semana['home_moneyline'].apply(moneyline_to_decimal)
df_semana['away_odds_decimal'] = df_semana['away_moneyline'].apply(moneyline_to_decimal)

# Adicionar coluna de previsão de vencedor
if 'predicted_winner' not in df_semana.columns:
    df_semana['predicted_winner'] = "Sem previsão"  # Inicializar a coluna, se não existir
else:
    df_semana['predicted_winner'] = df_semana['predicted_winner'].fillna("Sem previsão")

# Atribuir valores da previsão de acordo com o CSV
df_semana['predicted_winner'] = df_semana['game_id'].map(
    previsao.set_index('game_id')['predicted_winner']
)

# Calcular o lucro para cada jogo
df_semana['lucro'] = df_semana.apply(calcular_lucro, axis=1, valor_entrada=valor_entrada)

# Criar a tabela do Rich
table = Table(title=f"Comparação de Previsões para os Jogos da Semana {semana}")

# Adicionar colunas na tabela
table.add_column("Time da Casa", style="bold cyan")
table.add_column("Time Visitante", style="bold cyan")
table.add_column("Vencedor Previsto", style="bold magenta")
table.add_column("Vencedor Real", style="yellow")
table.add_column("Odds Casa", style="green")
table.add_column("Odds Visitante", style="red")
table.add_column("Lucro", style="bold green")

# Adicionar as linhas com os dados para cada jogo
for _, row in df_semana.iterrows():
    predicted_team = row['home_team'] if row['predicted_winner'] == 1 else (
        row['away_team'] if row['predicted_winner'] == 0 else "Sem previsão"
    )
    real_winner_team = (
        row['home_team'] if row['winner'] == 1 
        else row['away_team'] if row['winner'] == 0 
        else "Ainda não definido"
    )
    lucro = row['lucro'] if 'lucro' in row else 0
    
    table.add_row(
        row['home_team'],
        row['away_team'],
        predicted_team,
        real_winner_team,
        f"{row['home_odds_decimal']:.2f}",
        f"{row['away_odds_decimal']:.2f}",
        f"${lucro:.2f}"
    )

# Exibir a tabela
console.print(table)

# Calcular métricas da semana
acertos = (df_semana['predicted_winner'] == df_semana['winner']).sum()
total_jogos = len(df_semana[df_semana['winner'].notna()])  # Ignorar jogos sem resultado
assertividade = (acertos / total_jogos * 100) if total_jogos > 0 else 0
lucro_total = df_semana['lucro'].sum() if 'lucro' in df_semana else 0
porcentagem_lucro = (lucro_total / (total_jogos * valor_entrada)) * 100 if total_jogos > 0 else 0

# Exibir resultado geral da semana
result_table = Table(title="Resultados da Semana")
result_table.add_column("Métrica", style="bold cyan")
result_table.add_column("Valor", style="bold magenta")

result_table.add_row("Total de jogos", str(total_jogos))
result_table.add_row("Acertos", str(acertos))
result_table.add_row("Assertividade", f"{assertividade:.2f}%")
result_table.add_row("Lucro total", f"${lucro_total:.2f}")
result_table.add_row("Porcentagem de lucro", f"{porcentagem_lucro:.2f}%")

console.print(result_table)

Digite o número da semana (por exemplo, 1 para semana 1):  16
Digite o valor da entrada para cada jogo (exemplo: 100):  5


                            Comparação de Previsões para os Jogos da Semana 16                             
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Time da Casa ┃ Time Visitante ┃ Vencedor Previsto ┃ Vencedor Real ┃ Odds Casa ┃ Odds Visitante ┃ Lucro  ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ LAC          │ DEN            │ LAC               │ DEN           │ 1.65      │ 2.30           │ $-5.00 │
│ KC           │ HOU            │ KC                │ HOU           │ 2.10      │ 1.77           │ $-5.00 │
│ BAL          │ PIT            │ BAL               │ PIT           │ 1.35      │ 3.30           │ $-5.00 │
│ ATL          │ NYG            │ ATL               │ NYG           │ 1.24      │ 4.20           │ $-5.00 │
│ BUF          │ NE             │ BUF               │ NE            │ 1.11      │ 7.00           │ $-5.00 │
│ CAR          │ ARI            │ CAR               │ ARI           │ 2.64      │ 1.51           │ $-5.00 │
│ CHI          │ DET            │ CHI               │ DET           │ nan       │ 1.31           │ $-5.00 │
│ CIN          │ CLE            │ CIN               │ CLE           │ 1.31      │ 3.60           │ $-5.00 │
│ IND          │ TEN            │ TEN               │ TEN           │ 1.49      │ 2.70           │ $8.50  │
│ NYJ          │ LA             │ NYJ               │ LA            │ 2.50      │ 1.56           │ $-5.00 │
│ WAS          │ PHI            │ PHI               │ PHI           │ 2.36      │ 1.62           │ $3.09  │
│ SEA          │ MIN            │ SEA               │ MIN           │ 2.45      │ 1.57           │ $-5.00 │
│ LV           │ JAX            │ LV                │ JAX           │ 1.91      │ 1.91           │ $-5.00 │
│ MIA          │ SF             │ MIA               │ SF            │ 1.82      │ 2.02           │ $-5.00 │
│ DAL          │ TB             │ DAL               │ TB            │ 2.70      │ 1.49           │ $-5.00 │
│ GB           │ NO             │ GB                │ NO            │ 1.12      │ 6.50           │ $-5.00 │
└──────────────┴────────────────┴───────────────────┴───────────────┴───────────┴────────────────┴────────┘

       Resultados da Semana       
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Métrica              ┃ Valor   ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ Total de jogos       │ 16      │
│ Acertos              │ 2       │
│ Assertividade        │ 12.50%  │
│ Lucro total          │ $-58.41 │
│ Porcentagem de lucro │ -73.02% │
└──────────────────────┴─────────┘